In [1]:
import pandas as pd
import numpy as np

In [2]:
regions = pd.read_csv('economic_development.csv')[['Kod', 'Gmina', 'Powiat', 'Wojewodztwo']].drop_duplicates()

In [4]:
eu_funds = pd.DataFrame(columns=['Kod', 'Gmina', 'Powiat', 'Wojewodztwo', 'Rok', 'Dotacja'])

In [5]:
def random_dates(start, end, n=10):
    start_u = start.value//10**9
    end_u = end.value//10**9
    seen_dates = []
    for i in range(n):
        date = pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')[0]
        date_str = f"{date.year}-{date.month:02d}-{date.day:02d}"
        while date_str in seen_dates:
            date = pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')[0]
            date_str = f"{date.year}-{date.month:02d}-{date.day:02d}"
        seen_dates.append(date_str)
    return seen_dates

In [25]:
weights_v = {
    'POMORSKIE': 47.5,
    'ZACHODNIOPOMORSKIE': 37.7,
    'LUBUSKIE': 21.2,
    'WIELKOPOLSKIE': 52.9,
    'KUJAWSKO-POMORSKIE': 36.8,
    'MAZOWIECKIE': 106.3,
    'WARMIŃSKO-MAZURSKIE': 40.5,
    'PODLASKIE': 25,
    'DOLNOŚLĄSKIE': 53.1,
    'OPOLSKIE': 18,
    'ŁÓDZKIE': 54.2,
    'ŚLĄSKIE': 76.3,
    'ŚWIĘTOKRZYSKIE': 25.5,
    'MAŁOPOLSKIE': 61,
    'PODKARPACKIE': 48,
    'LUBELSKIE': 46.2,
}
weights_y = {
    2004: 2,
    2005: 4,
    2006: 5,
    2007: 7.5,
    2008: 7.5,
    2009: 9,
    2010: 11.5,
    2011: 14,
    2012: 15.5,
    2013: 15.5,
    2014: 17,
    2015: 13,
    2016: 10,
    2017: 11,
    2018: 16,
    2019: 16,
    2020: 13,
    2021: 13,
    2022: 13,
}
sum_weights_v = sum(list(weights_v.values()))
sum_weights_y = sum(list(weights_y.values()))
m_count = regions.groupby(by='Wojewodztwo')['Gmina'].count()

In [ ]:
eu_funds = pd.DataFrame(columns=['Kod', 'Gmina', 'Powiat', 'Wojewodztwo', 'Rok', 'Data', 'Dotacja'])
for i, row in regions.iterrows():
    if i % 10 == 0:
        print(i/len(regions))
    weight_v = weights_v[row['Wojewodztwo']] * 1e9 / m_count[row['Wojewodztwo']]
    for year in range(2004, 2023):
        weight_y = weights_y[year] / sum_weights_y
        mean_for_year = weight_v * weight_y
        n_dates = np.random.randint(1, 5)
        mean_per_donation = mean_for_year / n_dates
        donations = np.random.exponential(mean_per_donation - 1000, n_dates) + 1000 
        dates = random_dates(pd.to_datetime(f'{year}-01-01'), pd.to_datetime(f'{year+1}-01-01'), 1)
        for i, date in enumerate(dates):
            eu_funds.loc[len(eu_funds)] = {
                'Kod': row['Kod'],
                'Gmina': row['Gmina'],
                'Powiat': row['Powiat'],
                'Wojewodztwo': row['Wojewodztwo'],
                'Rok': year,
                'Data': date,
                'Dotacja': donations[i]
            }


In [41]:
eu_funds['Dotacja'] = eu_funds['Dotacja']*2

In [49]:
eu_funds['Dotacja'] = eu_funds['Dotacja'].astype(np.int32)

In [46]:
eu_funds.to_csv('eu_funds.csv', sep=',', index=False)